In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install -U sentence-transformers
!pip install evaluate
!pip install transformers[torch]

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hyyxxjug
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hyyxxjug
  Resolved https://github.com/huggingface/transformers.git to commit 35551f9a0f66a22de4971b4a51b3c172d3b87f95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8049176 sha256=0033800888c0b6e6a72ce077d7904b6dd5d9b5ac28144ea7cafa362c61c0671b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ogb7of7o/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# dataset
!gdown 1wb6ayDuhhqOnFLjU4qWzeohiMnv7t8RK

!gdown 1vzYpVcquBvzX5Ige3klpaACQFbjEP4Ak

!gdown 1yBppNyzNCS5tinBvlTIyuMbBDmQhmKBF
!gdown 1GvsfK3vZIBbYViI-KFPCsW-mFw4RUjqK

Downloading...
From: https://drive.google.com/uc?id=1wb6ayDuhhqOnFLjU4qWzeohiMnv7t8RK
To: /content/train_dataset_train.csv
100% 13.6M/13.6M [00:00<00:00, 24.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vzYpVcquBvzX5Ige3klpaACQFbjEP4Ak
To: /content/data_corrected_spell.csv
100% 13.2M/13.2M [00:00<00:00, 55.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yBppNyzNCS5tinBvlTIyuMbBDmQhmKBF
To: /content/label2id.json
100% 15.6k/15.6k [00:00<00:00, 42.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GvsfK3vZIBbYViI-KFPCsW-mFw4RUjqK
To: /content/id2label.json
100% 16.0k/16.0k [00:00<00:00, 40.5MB/s]


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import evaluate

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

import re

import json

In [ ]:
data = pd.read_csv('data_corrected_spell.csv')

### take part of dataset
# data = data.sample(data.shape[0] // 10)
# data = data.sample(20)

In [ ]:
def remove_extra_symbols(text):
    # Убираем лишние символы в начале предложения, если модель их добавила
    # correct_text = correct_text[correct_text.index(input_text[0]):]
    text = text.lstrip('.,[]«»')

    # Если модель выдает несколько одинаковых знаков препинания подряд, оставляем один
    text = re.sub(r'([^\w\s])\1+', r'\1', text)

    return text

data['Текст инцидента'] = data['Текст инцидента'].apply(remove_extra_symbols)

In [ ]:
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу


In [ ]:
def get_id_and_labels():
    id2label_path = 'id2label.json'
    label2id_path = 'label2id.json'

    with open(id2label_path, 'r', encoding='UTF-8') as file:
        id2label = json.load(file)

    id2label = {int(key):value for key,value in id2label.items()}

    with open(label2id_path, 'r', encoding='UTF-8') as file:
        label2id = json.load(file)

    return id2label, label2id

id2label, label2id = get_id_and_labels()

In [ ]:
data['label'] = [label2id[topic] for topic in data['Тема']]
data

,Исполнитель,Группа тем,Текст инцидента,Тема,label
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,0
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,1
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,2
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,3
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,4
...,...,...,...,...,...
22485,Министерство социального развития ПК,Социальное обслуживание и защита,"А если ещё не погасили ипотеку, но площадь бол...",Улучшение жилищных условий,125
22486,Губахинский городской округ,ЖКХ,Город Гремячинск — ситуация с теплом на улице ...,Ненадлежащее качество или отсутствие отопления,44
22487,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, у меня ребёнку 2 месяца. Тест на...",Технические проблемы с записью на прием к врачу,4
22488,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов?! Воо...,Благоустройство придомовых территорий,122


In [ ]:
# checkpoint = "xlm-roberta-base"
# checkpoint = "cointegrated/rubert-tiny2"
checkpoint = "ai-forever/ruBert-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels = data['Тема'].unique().shape[0],
    id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data, val_data = train_test_split(
    data[['Текст инцидента', 'label']], random_state=42, test_size=.1
)

In [ ]:
train_data

,Текст инцидента,label
13061,"Эти животные ни когда трубку не берите, либо с...",4
1166,Какое количество привитых граждан заболели пов...,47
20482,"Добрый день. Скажите, какие документы нужны дл...",2
5068,"Когда шла стройка ГРЕС, население в Яйве =1500...",40
7685,Прошу повлиять на УК «Губерния». Лёд и снег уп...,14
...,...,...
11964,Добрый день. С кем можно посоветоваться по пов...,57
21575,Добрый день! Подавала на выплату с 3 до 7 лет....,2
5390,"Это цифры не по Пермскому краю! Непонятно, поч...",52
860,"Глава района новый, видимо, ещё не в курсе, чт...",9


In [ ]:
class TextDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences = data_df["Текст инцидента"].values
        self.labels = data_df['label'].values

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, i):
        sentence, label = self.sentences[i], self.labels[i]

        tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=self.max_length)

        tokens['labels'] = label

        tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}

        # tokens['labels'] = tokens['labels'].to(torch.float)

        return tokens


train_dataset = TextDataset(train_data, tokenizer)
val_dataset = TextDataset(val_data, tokenizer)

# train_dataset[0]

In [ ]:
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    out = {}

    out.update(accuracy.compute(predictions=predictions, references=labels))
    out.update(f1_metric.compute(predictions=predictions, references=labels,
                                 average='weighted'))

    return out

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(None, classes=np.array(list(label2id.keys())), y=data["Тема"])

class_weights = torch.tensor(class_weights, device=model.device).to(torch.float).to("cuda")

In [ ]:
from torch import nn


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="promobot/models/ruBert-base",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.238800,2.142157,0.498888,0.429913
2,1.723300,1.960025,0.534460,0.475102


TrainOutput(global_step=10122, training_loss=2.270326754199354, metrics={'train_runtime': 4646.6196, 'train_samples_per_second': 8.712, 'train_steps_per_second': 2.178, 'total_flos': 1.0669718985689088e+16, 'train_loss': 2.270326754199354, 'epoch': 2.0})

In [ ]:
class_weights = compute_class_weight("balanced", classes=np.array(list(label2id.keys())), y=data["Тема"])

class_weights = torch.tensor(class_weights, device=model.device).to(torch.float).to("cuda")

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="promobot/models/ruBert-base",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.039700,2.565021,0.547799,0.520341


TrainOutput(global_step=5061, training_loss=2.20413623772881, metrics={'train_runtime': 2327.8108, 'train_samples_per_second': 8.695, 'train_steps_per_second': 2.174, 'total_flos': 5334859492844544.0, 'train_loss': 2.20413623772881, 'epoch': 1.0})

In [ ]:
model.save_pretrained("ruBert-base/")

In [ ]:
!zip -r ruBert-base.zip ruBert-base

  adding: ruBert-base/ (stored 0%)
  adding: ruBert-base/config.json (deflated 88%)
  adding: ruBert-base/model.safetensors (deflated 7%)


In [ ]:
!cp ruBert-base.zip /content/drive/MyDrive/

### try running it

In [ ]:
!gdown 1Uj_qnIfoxgMUuWF6_hwFN3yb3ILNXS8H

Downloading...
From: https://drive.google.com/uc?id=1Uj_qnIfoxgMUuWF6_hwFN3yb3ILNXS8H
To: /content/topic2big_topic.json
100% 21.4k/21.4k [00:00<00:00, 45.3MB/s]


In [ ]:
with open("topic2big_topic.json", "r", encoding="UTF8") as f:
    topic2big_topic = json.load(f)

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification",
                      model="ruBert-base",
                      tokenizer="ai-forever/ruBert-base")

In [ ]:
n = np.random.randint(data.shape[0])

print(n)

print(data["Текст инцидента"][n], )
print(data["Тема"][n], '|', data["Группа тем"][n])

label = classifier(data["Текст инцидента"][n])[0]["label"]

print(label, '|', topic2big_topic[label])

11394
В Голованово не только в частном секторе, но и среди 2-3 этажных домов тоже нет освещения. Сегодня бабушка шла вечером домой и в потемках упала в лужу. Темно очень.
Отсутствие фонарей освещения | Благоустройство
Освещение неисправно или отсутствует | Дороги


### test

In [ ]:
val_data

,Текст инцидента,label
4003,"'Здравствуйте,на против церкви на Чапаева не р...",127
3615,"'Здравствуйте!<br>Ввиду того, что на мосту Кам...",157
7907,'Уважаемая Администрация! Что опять на автобус...,87
1398,"'Здравствуйте, скажите пожалуйста более точнее...",1
12113,"'Здравствуйте,хотелось бы получить ответ на во...",22
...,...,...
13852,'А на детей с 8 лет каковы выплаты?,2
274,"'Здравствуйте ,как быть подскажите ,у меня 1/4...",5
14145,'Здравствуйте!,1
8465,"'Добрый день! Скажите пожалуйста, к кому обращ...",125


In [ ]:
n = np.random.randint(val_data.shape[0])
n

409

In [ ]:
sentence = str(val_data["Текст инцидента"].iloc[n])

print(sentence, '\n', id2label[val_data["label"].iloc[n]])

tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=512)

tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}


for key in tokens:
    tokens[key] = tokens[key].to("cuda").unsqueeze(0)

pred = model(**tokens)

'Добрый день. Проживаю в городе Лысьва в своём доме. За забором на перекрестке стоит колодец водоканала. Весной всегда он переполняется и вода под землёй поступает в наши колодцы. Каждый год звоню в водоканал, всегда приезжали и выкачивали. Сегодня сообщили что этот калодец частный, т.е. мой, на балансе водоканала не числится. Как так? Был их, стал мой. Разъясняющих ответов от руководства водоканала не услышала, просто бросили трубку. Хочу справедливости. Получается целая улица просто так пьёт воду больше 20 лет? 
 Подключение к водоснабжению


In [ ]:
id2label[pred["logits"].argmax().item()]

'★ Открытые канализационные люки'

# make submit

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install -U sentence-transformers
!pip install evaluate
!pip install transformers[torch]

In [ ]:
# model
!gdown 1-QL4U5d3gO4U1RoWhnTCDfSx2mSz-4Vb
!unzip classification_model.zip

In [ ]:
# dataset
!gdown 1-vKglR5qwkmXGT8EK2La2eiNy1ApKLPD
!unzip merged_datasets4nlp.zip
!mv merged_datasets4nlp dataset

Downloading...
From: https://drive.google.com/uc?id=1-vKglR5qwkmXGT8EK2La2eiNy1ApKLPD
To: /content/merged_datasets4nlp.zip
100% 225M/225M [00:02<00:00, 97.1MB/s]
Archive:  merged_datasets4nlp.zip
   creating: merged_datasets4nlp/
  inflating: merged_datasets4nlp/merged_test4nlp.parquet  
  inflating: merged_datasets4nlp/merged_train4nlp.parquet  


## Making Predictions for Catboost Training

### Making Train

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

import evaluate

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

In [ ]:
checkpoint = "models/regression_model/checkpoint-34486"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
data = pd.read_parquet('datasets/train_full_text_w_diff.parquet')

In [ ]:
# data = pd.read_parquet("datasets/merged_test4nlp.parquet")
# data = data.sample(data.shape[0] // 5)

In [ ]:
data

,variantid1,variantid2,full_text_1,full_text_2
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны..."
2,77243372,479860557,name: Цифровой кабель TV-COM HDMI 1.4 (M/ M) F...,name: Кабель HDMI 1.4 (Male/Male) (CG150S-1.5...
3,86065820,540678372,name: Игровая мышь проводная A4Tech Bloody P93...,"name: Мышь A4Tech Bloody P93s Bullet, серый, о..."
4,91566575,258840506,"name: Смартфон Vivo Y93 1815 3/32 ГБ, черный\n...","name: Смартфон Vivo Y81 3/32 ГБ, черный\ncolor..."
...,...,...,...,...
18079,666998614,667074522,name: Оперативная память Corsair VENGEANCE LPX...,name: Оперативная память Corsair VENGEANCE LPX...
18080,670036240,670048449,name: Зарядное устройство для HP 15-bw661ur бл...,name: Зарядное устройство для HP 15s-fq2066ur ...
18081,670284509,684323809,name: Ремешок из натуральной кожи 20 мм для Sa...,name: Ремешок для часов UTHAI Z08 plus из нату...
18082,692172005,704805270,name: Лазерный новогодний проектор для дома и ...,name: Новогодний проектор гирлянда морозостойк...


In [ ]:
class TextTestDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences1 = data_df["full_text_1"].values
        self.sentences2 = data_df['full_text_2'].values
        # self.labels = data_df['target'].values

    def __len__(self):
        return self.sentences1.shape[0]

    def __getitem__(self, i):
        sentence1, sentence2 = self.sentences1[i], self.sentences2[i]

        tokens = tokenizer(sentence1, sentence2, truncation="longest_first", padding="max_length", max_length=self.max_length)

        # labels = [0, 0]

        # labels[int(label)] = 1

        # tokens['labels'] = label

        tokens = {key: torch.tensor(val) for key, val in tokens.items()}

        return tokens


test_dataset = TextTestDataset(data, tokenizer)

test_dataset[0]

{'input_ids': tensor([     0,   9351,     12,  65629,  99937,  57213, 155616,   1091,   1219,
          33591,    908,   2242,     31, 101715,   9513,  12988,  38274,   4265,
          87005,   2256,    209,  99657,   7515,    363,     12,  65629,  99937,
          16051,     35, 238925,    227,   7515,    617,     12,  65629,  99937,
          57213,    672, 146835,  59650,      4,    417,     12,  22410,   1537,
          79654,     12,    908,   2242,     31, 108551,     12,  65629,  99937,
          57213, 112383,      9,  10655,  29692,  91431,   3429,     89,     12,
         155616,  34504,    130,  49517,  16258,     12,  20992,    547, 144854,
         107722,     59,      9,   4135,  29864,  63354,     12,  57592, 223392,
            983,   2192,  19011,     12,   1936,   6365,     15,    650,  90053,
          55034,   4544,  26623,   1270,  20999,     35,     84,  45580, 104454,
             16,    672,   5642,  66639,     12,      6, 164057,    650,  50560,
         222377

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
model = model.to("cuda")

In [ ]:
all_preds = []

with torch.no_grad():
    for tokens in tqdm(test_dataloader):
        for key in tokens:
            tokens[key] = tokens[key].to("cuda")

        pred = model(**tokens)
        pred = pred['logits'].detach().cpu().numpy().tolist()

        all_preds.extend(pred)

  0%|          | 0/283 [00:00<?, ?it/s]

In [ ]:
all_preds

[[0.021984655410051346],
 [0.040481965988874435],
 [0.06488964706659317],
 [0.7282276749610901],
 [0.18905960023403168],
 [0.9768507480621338],
 [0.9621462821960449],
 [1.0194969177246094],
 [0.9036983251571655],
 [0.7330847978591919],
 [0.8377479314804077],
 [0.0661676749587059],
 [0.05389318987727165],
 [0.10925476998090744],
 [0.976178765296936],
 [0.7431394457817078],
 [0.24526426196098328],
 [0.9815181493759155],
 [0.016352642327547073],
 [0.03327435627579689],
 [0.5078286528587341],
 [0.7759946584701538],
 [0.10535780340433121],
 [0.2217760682106018],
 [0.03893611952662468],
 [0.7499988675117493],
 [0.0725361630320549],
 [0.1273512840270996],
 [0.04803674295544624],
 [0.9799312353134155],
 [0.03721999749541283],
 [0.24384421110153198],
 [0.9785986542701721],
 [0.13758867979049683],
 [0.08147022873163223],
 [0.09958720952272415],
 [0.8956090807914734],
 [0.01893923059105873],
 [0.0893750935792923],
 [0.048545632511377335],
 [0.8075284957885742],
 [0.8835874795913696],
 [0.10606766

In [ ]:
all_preds[-1]

[0.6095085144042969]

In [ ]:
np.array(all_preds).flatten().shape

(18084,)

In [ ]:
# data['target'] = [i[0] if i[0] <= 1 else 1 for i in all_preds]
data['target'] = np.array(all_preds).flatten()

In [ ]:
data = data.drop(['full_text_1', 'full_text_2'], axis=1)
data

KeyError: "['full_text_1', 'full_text_2'] not found in axis"

In [ ]:
data['target'].max(), data['target'].min(), data['target'].mean()

(1.0375804901123047, 0.011065545491874218, 0.3859469411834218)

In [ ]:
data.to_csv('submits/submit_xlm_roberta_1_epoch.csv')

---

### Test

In [ ]:
# data = pd.read_parquet("datasets/merged_test4nlp.parquet")
data = pd.read_parquet('datasets/test_full_text_w_diff.parquet')

In [ ]:
data

,variantid1,variantid2,full_text_1,full_text_2
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны..."
2,77243372,479860557,name: Цифровой кабель TV-COM HDMI 1.4 (M/ M) F...,name: Кабель HDMI 1.4 (Male/Male) (CG150S-1.5...
3,86065820,540678372,name: Игровая мышь проводная A4Tech Bloody P93...,"name: Мышь A4Tech Bloody P93s Bullet, серый, о..."
4,91566575,258840506,"name: Смартфон Vivo Y93 1815 3/32 ГБ, черный\n...","name: Смартфон Vivo Y81 3/32 ГБ, черный\ncolor..."
...,...,...,...,...
18079,666998614,667074522,name: Оперативная память Corsair VENGEANCE LPX...,name: Оперативная память Corsair VENGEANCE LPX...
18080,670036240,670048449,name: Зарядное устройство для HP 15-bw661ur бл...,name: Зарядное устройство для HP 15s-fq2066ur ...
18081,670284509,684323809,name: Ремешок из натуральной кожи 20 мм для Sa...,name: Ремешок для часов UTHAI Z08 plus из нату...
18082,692172005,704805270,name: Лазерный новогодний проектор для дома и ...,name: Новогодний проектор гирлянда морозостойк...


In [ ]:
class TextTestDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences1 = data_df["full_text_1_w_diff"].values
        self.sentences2 = data_df['full_text_2_w_diff'].values
        # self.labels = data_df['target'].values

    def __len__(self):
        return self.sentences1.shape[0]

    def __getitem__(self, i):
        sentence1, sentence2 = self.sentences1[i], self.sentences2[i]

        tokens = tokenizer(sentence1, sentence2, truncation="longest_first", padding="max_length", max_length=self.max_length)

        # labels = [0, 0]

        # labels[int(label)] = 1

        # tokens['labels'] = label

        tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}

        return tokens


test_dataset = TextTestDataset(data, tokenizer)

test_dataset[0]

{'input_ids': tensor([   101,  90377,    162,  21008,    657,  91987,    435,  79992,   2844,
          35185,  88313,    436,    187,    440,    415,    439,    197,   1355,
          29553,  88850,  10525,  15040,  25679,   1355,  26427,  95852,    439,
            162,  21008,    676,    107,  76375,  95852,    442,    162,  21008,
            657,  18680,    121,    113,    162,    141,    126,    167,  29729,
            162,  35185,  88313,    436,  12097,    162,  21008,    657,  15197,
            133,  18194,  21008,    162,  91987,    435, 118686,    378,  12097,
            162,  79992,    942,    378,   6471,    133,  51123,    749,    162,
          29519,    378,  20434,    667,    378,   4846,    162,   3510,   1336,
            160,    110,   9387,  46311,  70356,   7491,    107,  53650,    158,
          17817,    162,  85114,    394,    667,    378,   3414,    113,  53792,
            121,  26427,    162,   1355,  26832,    977,    162,  21008,    657,
            121

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
all_preds = []

with torch.no_grad():

    for tokens in tqdm(test_dataloader):

        for key in tokens:
            tokens[key] = tokens[key].to("cuda")

        pred = model(**tokens)

        pred = pred['logits'].detach().cpu().numpy().tolist()

        all_preds.extend(pred)

  0%|          | 0/283 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
all_preds

[[2.2426047325134277, -1.9304583072662354],
 [2.179056406021118, -1.8325953483581543],
 [1.8400065898895264, -1.5619735717773438],
 [0.92543625831604, -0.7615549564361572],
 [0.7971125841140747, -0.6422891020774841],
 [-2.1143338680267334, 1.8639883995056152],
 [-1.9176989793777466, 1.7153599262237549],
 [-2.369076728820801, 1.9813315868377686],
 [-0.9303659796714783, 0.9066335558891296],
 [-0.8769173622131348, 0.840950071811676],
 [-0.5726568102836609, 0.5440670251846313],
 [2.186034679412842, -1.8472533226013184],
 [2.1729559898376465, -1.8362997770309448],
 [0.9580971002578735, -0.8351223468780518],
 [-1.8961281776428223, 1.715293526649475],
 [-1.6053791046142578, 1.4456579685211182],
 [-1.8610889911651611, 1.7069562673568726],
 [0.25287288427352905, -0.21320819854736328],
 [2.2502198219299316, -1.9365403652191162],
 [2.188000202178955, -1.8490524291992188],
 [-0.0656939297914505, 0.20833264291286469],
 [-0.0344240628182888, 0.16713164746761322],
 [1.046304702758789, -0.790191769599

In [ ]:
data['xlm_roberta_pred_1'] = np.array([i for i in all_preds])[:,0]
data['xlm_roberta_pred_2'] = np.array([i for i in all_preds])[:,1]

data = data.drop(['full_text_1_w_diff', 'full_text_2_w_diff'], axis=1)
data

In [ ]:
data.head(2)

,variantid1,variantid2,full_text_1,full_text_2,classification_pred
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...,"[2.2426047325134277, -1.9304583072662354]"
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны...","[2.179056406021118, -1.8325953483581543]"


In [ ]:
data.to_csv('datasets/test_xlm_roberta_class_full_text_w_diff.csv')